In [6]:
#%save ExempleMySql.py _ih [4]

In [1]:
import mysql.connector
from mysql.connector import errorcode





class monSql :
    def __init__ (self, 
                 http = "193.70.14.209",
                 base = "db_chatbot", 
                 user = "db_chatbot",
                  port = 3308,
                 password =  "db_chatbot_pwd",
                 ) :
        self.cnx = mysql.connector.connect(user=user, password=password,
                                           host=http, port=port,
                                           )
        
        self.cursor = self.cnx.cursor()
        
        # on regarde si la base existe et  sinon on cree
        try:
            self.cnx.database = base
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_BAD_DB_ERROR:
                self.create_database(base)
                self.cnx.database = base
            else:
                raise
        
        self.messageUpdate = (
        "UPDATE tableaux SET value =%(value)f "
        "WHERE "
        "client ='%(client)s' AND " 
        " type_data ='%(type_data)s' AND "
        " period =%(period)i AND "
        " row =%(row)i AND "
        " col =%(col)i "
        
        )
        
        self.messageInsert = (
        "INSERT into `tableaux` "
        " (`client`, `type_data`, `period` , `row`, `col`, `value` )"
        " VALUES  ("
        "'%(client)s', '%(type_data)s', %(period)i, %(row)i, %(col)i, %(value)f )"
        
        )
        
        self.messageDelete = (
        "DELETE FROM tableaux "
        "WHERE "
        "client ='%(client)s' AND " 
        "type_data ='%(type_data)s' AND "
        "period =%(period)i "
        )
        
        self.messageDelete2 = (
        "DELETE FROM tableaux "
        "WHERE "
        "client ='%(client)s' " 
        )
        
        self.messageSelect1 = (
        "SELECT * FROM tableaux "
        "WHERE "
        "client ='%(client)s' AND " 
        "type_data ='%(type_data)s' AND "
        "period =%(period)i "
        )
        
        self.ordreData = ['ID',
                          'client',
                         'type_data',
                         'period',
                          'row',
                          'col',
                          'value',
                         ]
    
    
    def create_database(self, base):
        try:
            self.cursor.execute(
                "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(base))
        except mysql.connector.Error as err:
            print("Failed creating database: {}".format(err))
            raise
        return
             
    #methode qui ferme la connexion avec une base    
    def close (self,) :
        self.cnx.close()
        
    def create_table ( self, 
                      http = "127.0.0.1",
                      base = "essai",
                      user = 'root',
                      password  = 'root',
                      isNew  = False,
                  
                      ) :
        
        #tables c'est un dictionnaire contenant le nom des tables en clé et leurs descriptions en valeurs
    
        tables = {}




        tables ["tableaux"] = (
            "CREATE TABLE `tableaux`  ("
            "`ID` bigint (32) AUTO_INCREMENT,"
             " `client` varchar(255) NOT NULL," 
             " `type_data` varchar(255) NOT NULL," 
             " `period` int(32) ," 
             " `row` int (8) ,"
             " `col` int (8) ,"
             " `value` double, "
            " PRIMARY KEY (`ID`)"
             ") ENGINE=InnoDB"    )


        for name, ddl in tables.items():
            if isNew :
                command = "DROP TABLE IF EXISTs " + name
                try :
                    print ("Dropping table {}  =>".format(name))
                    self.cursor.execute(command)
                except:
                    print (" error during drop ?")
                    raise
                else:
                    print (" Drop ok")
                    
            try:
                print("Creating table {}: ".format(name))
                self.cursor.execute(ddl)
            except mysql.connector.Error as err:
                if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                    print("already exists.")
                else:
                    print(err.msg)
                    raise
            else:
                print("OK")
            continue
        return
    
    def insertMatrice (self, datas) :
        
        for dico in datas :
            
            m = self.messageInsert%dico
            
            print (m)
            
            try :
                self.cursor.execute(m)
            except:
                self.cnx.rollback()
                self.cnx.close ()
                raise
            
            
            continue
            
        self.cnx.commit()
        return
    
     # pour detruire toutes les matrices d'un client 
    def delete_all (self, client ):
        m = self.messageDelete2%{"client" : client}
        
        try:
            # Execute the SQL delete
            self.cursor.execute(m)
        except:
            print ("Error: unable to delete matrix data =", m)
            self.cnx.rollback()
            self.cnx.close()
            raise
        self.cnx.commit()
        return
    
    def _select (self, m) :
        
        rr = []
        try:
            # Execute the SQL command
            self.cursor.execute(m)
            # Fetch all the rows in a list of lists.
            results = self.cursor.fetchall()
        except:
            print ("Error: unable to fecth data")
            raise
        for row in results:
            r = {self.ordreData[i] : row[i] for i in range(0, len(row))  }
            rr.append(r)
            continue
            
        return rr   
        
    def selectMatrice (self, dico) :
        m = self.messageSelect1%dico
        return self._select(m)
                                           
                                        

    
if __name__ == "__main__" :
    print ("on y va")
    B = monSql(base = "db_chatbot")
    #creer la table tableaux
    B.create_table(isNew  = True)
    data = [{'period': -84, 'value': 4, 'client': 'essai', 'col': 4, 'type_data': 'VOLUME', 'row': 0}, 
            {'period': -84, 'value': 4, 'client': 'essai', 'col': 3, 'type_data': 'VOLUME', 'row': 0}, 
            {'period': -81, 'value': 40, 'client': 'essai02', 'col': 4, 'type_data': 'VOLUME', 'row': 0}]
    B.insertMatrice(data)
    dico = {'client': 'essai', 'type_data': 'VOLUME', 'period': -84}
    #affichage des elements de la table tableaux avec le client essai le "type data" volume et la period -84
    print (B.selectMatrice(dico))
    B.delete_all ("essai02")
    #
    B.close()
    print ("end_of_job")

on y va
Dropping table tableaux  =>
 Drop ok
Creating table tableaux: 
OK
INSERT into `tableaux`  (`client`, `type_data`, `period` , `row`, `col`, `value` ) VALUES  ('essai', 'VOLUME', -84, 0, 4, 4.000000 )
INSERT into `tableaux`  (`client`, `type_data`, `period` , `row`, `col`, `value` ) VALUES  ('essai', 'VOLUME', -84, 0, 3, 4.000000 )
INSERT into `tableaux`  (`client`, `type_data`, `period` , `row`, `col`, `value` ) VALUES  ('essai02', 'VOLUME', -81, 0, 4, 40.000000 )
[{'period': -84, 'value': 4.0, 'ID': 1, 'client': u'essai', 'col': 4, 'type_data': u'VOLUME', 'row': 0}, {'period': -84, 'value': 4.0, 'ID': 2, 'client': u'essai', 'col': 3, 'type_data': u'VOLUME', 'row': 0}]
end_of_job
